# Read Data

In [10]:
import numpy as np
import pandas as pd

In [11]:
train_balanced = pd.read_pickle('train_balanced.pkl')
val_balanced = pd.read_pickle('val_balanced.pkl')
test_balanced = pd.read_pickle('test_balanced.pkl')
VOCAB_BALANCED_SIZE = set()
for tokenized in train_balanced.tokenized:
    for token in tokenized:
        VOCAB_BALANCED_SIZE.add(token)
max(VOCAB_BALANCED_SIZE)     

7130

In [12]:
train_unbalanced = pd.read_pickle('train_unbalanced.pkl')
val_unbalanced = pd.read_pickle('val_unbalanced.pkl')
test_unbalanced = pd.read_pickle('test_unbalanced.pkl')

VOCAB_UNBALANCED_SIZE = set()
for tokenized in train_unbalanced.tokenized:
    for token in tokenized:
        VOCAB_UNBALANCED_SIZE.add(token)

max(VOCAB_UNBALANCED_SIZE)

6767

In [13]:
from datasets import Dataset

In [14]:
train_dataset_unbalanced = Dataset.from_dict({"input_ids": [x['input_ids'] for x in train_unbalanced.marbert_tokens], 
                                   "label": train_unbalanced['class'], 
                                   'attention_mask':[x['attention_mask'] for x in train_unbalanced.marbert_tokens]})
val_dataset_unbalanced = Dataset.from_dict({"input_ids": [x['input_ids'] for x in val_unbalanced.marbert_tokens], 
                                 "label": val_unbalanced['class'], 
                                 'attention_mask':[x['attention_mask'] for x in val_unbalanced.marbert_tokens]})
test_dataset_unbalanced = Dataset.from_dict({"input_ids": [x['input_ids'] for x in test_unbalanced.marbert_tokens], 
                                  "label": test_unbalanced['class'], 
                                  'attention_mask':[x['attention_mask'] for x in test_unbalanced.marbert_tokens]})

In [15]:
train_dataset_balanced = Dataset.from_dict({"input_ids": [x['input_ids'] for x in train_balanced.marbert_tokens], 
                                   "label": train_balanced['class'], 
                                   'attention_mask':[x['attention_mask'] for x in train_balanced.marbert_tokens]})
val_dataset_balanced = Dataset.from_dict({"input_ids": [x['input_ids'] for x in val_balanced.marbert_tokens], 
                                 "label": val_balanced['class'], 
                                 'attention_mask':[x['attention_mask'] for x in val_balanced.marbert_tokens]})
test_dataset_balanced = Dataset.from_dict({"input_ids": [x['input_ids'] for x in test_balanced.marbert_tokens], 
                                  "label": test_balanced['class'], 
                                  'attention_mask':[x['attention_mask'] for x in test_balanced.marbert_tokens]})

In [16]:
import transformers
from transformers import AutoModelForSequenceClassification

In [24]:
model_balanced = AutoModelForSequenceClassification.from_pretrained("UBC-NLP/MARBERTv2", num_labels=4)
for param in model_balanced.parameters(): param.data = param.data.contiguous()

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at UBC-NLP/MARBERTv2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [25]:
model_unbalanced = AutoModelForSequenceClassification.from_pretrained("UBC-NLP/MARBERTv2", num_labels=4)
for param in model_unbalanced.parameters(): param.data = param.data.contiguous()

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at UBC-NLP/MARBERTv2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [27]:
import numpy as np
import evaluate

metric = evaluate.load("f1")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    # convert the logits to their predicted class
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels, average='weighted')


In [31]:
from transformers import TrainingArguments, Trainer
import accelerate
training_args = TrainingArguments(
    output_dir="./results",
    logging_strategy='steps',
    logging_steps=10,
    metric_for_best_model='eval_f1',
    per_device_eval_batch_size=32,
    per_device_train_batch_size=32,
    load_best_model_at_end=True,
    eval_strategy="epoch",
    learning_rate=2e-5,
    num_train_epochs=5,
    save_strategy='epoch',
    greater_is_better=True,
    
)
trainer_balanced = Trainer(
    model=model_balanced,
    args=training_args,
    train_dataset=train_dataset_balanced,
    eval_dataset=val_dataset_balanced,
    compute_metrics=compute_metrics,
)
trainer_balanced.train()

Epoch,Training Loss,Validation Loss,F1
1,0.402500,0.701418,0.747460
2,0.343500,0.709247,0.773999
3,0.346100,0.781760,0.750801
4,0.275600,0.790377,0.765305
5,0.297600,0.783616,0.761926


TrainOutput(global_step=400, training_loss=0.3200368320941925, metrics={'train_runtime': 269.4293, 'train_samples_per_second': 47.248, 'train_steps_per_second': 1.485, 'total_flos': 1308384328368000.0, 'train_loss': 0.3200368320941925, 'epoch': 5.0})

In [30]:
from transformers import TrainingArguments, Trainer
import accelerate
training_args = TrainingArguments(
    output_dir="./results",
    logging_strategy='steps',
    logging_steps=10,
    metric_for_best_model='eval_f1',
    per_device_eval_batch_size=32,
    per_device_train_batch_size=32,
    load_best_model_at_end=True,
    eval_strategy="epoch",
    learning_rate=2e-5,
    num_train_epochs=5,
    save_strategy='epoch',
    greater_is_better=True,
    
)
trainer_unbalanced = Trainer(
    model=model_unbalanced,
    args=training_args,
    train_dataset=train_dataset_unbalanced,
    eval_dataset=val_dataset_unbalanced,
    compute_metrics=compute_metrics,
)
trainer_unbalanced.train()

Epoch,Training Loss,Validation Loss,F1
1,0.914300,0.837041,0.556045
2,0.687400,0.651385,0.744844
3,0.515500,0.594943,0.770123
4,0.400400,0.587355,0.767777
5,0.284500,0.595500,0.756163


TrainOutput(global_step=300, training_loss=0.6079052750269572, metrics={'train_runtime': 231.6059, 'train_samples_per_second': 41.147, 'train_steps_per_second': 1.295, 'total_flos': 979489603248000.0, 'train_loss': 0.6079052750269572, 'epoch': 5.0})

In [35]:
from sklearn.metrics import classification_report, confusion_matrix
from tqdm import tqdm
import torch
def test(trainer, test_dataset):
    preds=[]
    true=[]
    model = trainer.model
    model.eval()
    with torch.no_grad():
        for sample in tqdm(test_dataset):
            input_ids=torch.tensor(sample['input_ids'], dtype=torch.long).unsqueeze(0).to('cuda')
            attention_mask=torch.tensor(sample['attention_mask'], dtype=torch.long).unsqueeze(0).to('cuda')
            output = model(input_ids=input_ids, attention_mask=attention_mask)
            pred = torch.argmax(output.logits, dim=1).item()
            preds.append(pred)
            true.append(sample['label'])
    print(classification_report(y_pred=preds, y_true=true))
    print(confusion_matrix(y_pred=preds, y_true=true))

In [36]:
test(trainer_balanced, test_dataset_balanced)

100%|████████████████████████████████████████████████████████████████████████████████| 561/561 [00:05<00:00, 104.16it/s]

              precision    recall  f1-score   support

           0       0.78      0.72      0.75       148
           1       0.61      0.70      0.65        82
           2       0.72      0.56      0.63        75
           3       0.79      0.84      0.82       256

    accuracy                           0.75       561
   macro avg       0.73      0.71      0.71       561
weighted avg       0.75      0.75      0.75       561

[[107  10   6  25]
 [  7  57   3  15]
 [ 10   7  42  16]
 [ 14  19   7 216]]


In [37]:
test(trainer_unbalanced, test_dataset_unbalanced)

100%|████████████████████████████████████████████████████████████████████████████████| 561/561 [00:05<00:00, 101.81it/s]

              precision    recall  f1-score   support

           0       0.73      0.75      0.74       148
           1       0.76      0.67      0.71        82
           2       0.74      0.45      0.56        75
           3       0.77      0.88      0.82       256

    accuracy                           0.76       561
   macro avg       0.75      0.69      0.71       561
weighted avg       0.76      0.76      0.75       561

[[111   5   4  28]
 [  8  55   1  18]
 [ 16   5  34  20]
 [ 17   7   7 225]]
